# AP(P)3RO PROJECT (model de démo) #


#### M. Le Gras, A. Naullet, A. Calixte, P. Beaunieux ####

## Partie 1: Cartographie de la ville par le drone ##

On simule une ville avec un petit graphe

In [19]:
import numpy as np

In [1]:
#voici un graph simulant un morceau de ville
#(node1,node2, distance)                                                                                                                                 
G = [(1,2,29), (2,3,13), (3,4,21), (4,8,10), (8,2,2), (8,5,20), (8,6,12), (6,7,6), (6,1,22)]
print(G)

[(1, 2, 29), (2, 3, 13), (3, 4, 21), (4, 8, 10), (8, 2, 2), (8, 5, 20), (8, 6, 12), (6, 7, 6), (6, 1, 22)]


<img src="assets/img/graph_demo.png" width="400">

### 1.2 Calcul du parcours le plus efficace pour le drone ###

L'idée c'est de trouver un chemin eulerien, cependant notre graphe n'est pas eulerien
Nous allons alors utilisé un algo qui transforme un graph non-eulerien en un graph eulerien (sans altéré ses propriétés). Enfin on pourra utiliser un algo qui trouve un chemin eulerien c-a-d un chemin qui passe par toutes les arretes du graphes (simulant les rues de la ville)

In [18]:
#ICI on fait l'algo du cycle_eulerien_avec_erreur

In [ ]:
#fix le graph vers un graph eulerien


In [6]:
def odd_vertices(n, edges):
    deg = [0] * n
    for (a,b) in edges:
        deg[a] += 1
        deg[b] += 1
    return [a for a in range(n) if deg[a] % 2]

def is_edge_connected(n, edges):
    if n == 0 or len(edges) == 0:
        return True
    # Convert to adjacency list
    succ = [[] for a in range(n)]
    for (a,b) in edges:
        succ[a].append(b)
        succ[b].append(a)
    # BFS over the graph, starting from one extremity of the first edge
    touched = [False] * n
    init = edges[0][0]
    touched[init] = True
    todo = [init]
    while todo:
        s = todo.pop()
        for d in succ[s]:
            if touched[d]:
                continue
            touched[d] = True
            todo.append(d)
    for a in range(n):
        if succ[a] and not touched[a]:
            return False
    return True

def is_eulerian(n, edges):
    return is_edge_connected(n, edges) and not odd_vertices(n, edges)

def find_eulerian_cycle(n, edges):
    assert is_eulerian(n, edges)
    if len(edges) == 0:
        return []
    cycle = [edges[0][0]] # start somewhere
    while True:
        rest = []
        for (a, b) in edges:
            if cycle[-1] == a:
                cycle.append(b)
            elif cycle[-1] == b:
                cycle.append(a)
            else:
                rest.append((a,b))
        if not rest:
            assert cycle[0] == cycle[-1]
            return cycle[0:-1]
        edges = rest
        if cycle[0] == cycle[-1]:
            # Rotate the cycle so that the last state
            # has some outgoing edge in EDGES.
            for (a, b) in edges:
                if a in cycle:
                    idx = cycle.index(a)
                    cycle = cycle[idx:-1] + cycle[0:idx+1]
                    break
print(find_eulerian_cycle(len(G), G))

ValueError: too many values to unpack (expected 2)

In [3]:
# ICI ON AFFICHE le parcours

### 1.3 Recuperation des données du drones (simulation d'eneigement) ###

In [20]:
#ICI on simule l'eneigement (baill de plustard)